In [ ]:
#connect drive for files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#enter the path to directory
%cd /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts

/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/nlp


In [ ]:
#install all requirements
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/Shareddrives/ECE%20226%20Project/archai-neurips-lts/archai-neurips-lts
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/NVIDIA/dllogger.git to /tmp/pip-install-zvv7mugn/dllogger_1d0979c773b643b69cc730509a413723
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/dllogger.git /tmp/pip-install-zvv7mugn/dllogger_1d0979c773b643b69cc730509a413723
  Resolved https://github.com/NVIDIA/dllogger.git to commit 0540a43971f4a8a16693a9de9de73c1072020769
  Preparing metadata (setup.py) ... done
  Attempting uninstall: archai
    Found existing installation: archai 0.6.0
    Uninstalling archai-0.6.0:
      Successfully uninstalled archai-0.6.0
  Running setup.py develop for archai


In [ ]:
#run search for opt with param limits
!python archai/nlp/search.py --n_iter 30 --model_type hf_opt --n_layer 2 3 4 5 6 7 8 --n_head 2 4 8 --div_val 4 --param_constraint_lower 25000 --param_constraint_upper 10000000 --device cuda --device_name titanxp


baseline - finding the baseline architecture latency memory and decoder params

In [ ]:
!python archai/nlp/search.py --profile_baseline --model_type hf_opt --n_layer 2 3 4 5 6 7 8 --n_head 2 4 8 --div_val 4 --param_constraint_lower 25000 --param_constraint_upper 10000000 --device cuda --device_name titanxp --dataset wt2

2023-06-14 16:48:05.437220: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
STAGE:2023-06-14 16:49:46 4770:4770 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-06-14 16:49:46 4770:4770 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-06-14 16:49:46 4770:4770 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
config_0_j1 has: 
            263168 decoder_params
            8467968 total_params
            0.0030s latency
            69.6997MB memory
STAGE:2023-06-14 16:49:46 4770:4770 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-06-14 16:49:46 4770:4770 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-06-14 16:49:46 4770:4770 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
config_0_j2 has: 
            526336 decoder_params
            8731136 total_params
            0.0043s latency
            88.46

select pareto which are closest to the baseline

In [ ]:
!python archai/nlp/search.py --select_pareto --model_type hf_opt --n_layer 2 3 4 5 6 7 8 --n_head 2 4 8 --div_val 4 --param_constraint_lower 25000 --param_constraint_upper 10000000 --device cuda --device_name titanxp --dataset wt2

2023-06-14 16:50:24.190922: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Selected 17 of pareto jobs based on their proximity to the baseline architectures.
Unique Pareto-frontier jobs: 17


Training pareto architectures (will take time) change the path here. once trained, move the generated files in logdir to logdir/hf_opt_titanxp_3d/pareto/

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j0  --n_layer 2 --d_model 192 --d_inner 768 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-15 16:45:00.471236: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j0
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j0', experiment_name='config_0_j0', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j1  --n_layer 2 --d_model 704 --d_inner 1216 --n_head 8 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-15 16:49:47.219485: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j1
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j1', experiment_name='config_0_j1', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j2  --n_layer 2 --d_model 128 --d_inner 576 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-15 16:57:03.109379: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j2
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j2', experiment_name='config_0_j2', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j3  --n_layer 2 --d_model 640 --d_inner 1792 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-15 17:00:23.931793: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j3
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j3', experiment_name='config_0_j3', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j4  --n_layer 5 --d_model 448 --d_inner 768 --n_head 8 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-15 17:07:37.737820: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j4
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j4', experiment_name='config_0_j4', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j5  --n_layer 2 --d_model 256 --d_inner 448 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-15 17:12:41.670376: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j5
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j5', experiment_name='config_0_j5', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j6  --n_layer 2 --d_model 256 --d_inner 512 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-15 17:16:27.761725: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j6
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j6', experiment_name='config_0_j6', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j7  --n_layer 2 --d_model 640 --d_inner 1792 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-15 17:20:17.660494: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j7
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j7', experiment_name='config_0_j7', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j8  --n_layer 2 --d_model 512 --d_inner 1088 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-15 17:27:36.384529: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j8
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j8', experiment_name='config_0_j8', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j9  --n_layer 2 --d_model 128 --d_inner 256 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-15 17:33:12.582287: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j9
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j9', experiment_name='config_0_j9', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j10  --n_layer 2 --d_model 512 --d_inner 960 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-15 17:36:33.145125: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j10
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j10', experiment_name='config_0_j10', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j11  --n_layer 2 --d_model 128 --d_inner 320 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-15 17:41:54.116219: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j11
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j11', experiment_name='config_0_j11', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j12  --n_layer 2 --d_model 128 --d_inner 3264 --n_head 8 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-15 17:53:54.963893: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j12
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j12', experiment_name='config_0_j12', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j13  --n_layer 2 --d_model 640 --d_inner 1792 --n_head 8 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-15 17:57:48.145271: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j13
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j13', experiment_name='config_0_j13', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j14  --n_layer 2 --d_model 448 --d_inner 1152 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-15 18:04:40.689690: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j14
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j14', experiment_name='config_0_j14', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j15  --n_layer 2 --d_model 448 --d_inner 832 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-15 18:09:30.513451: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j15
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j15', experiment_name='config_0_j15', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j16  --n_layer 2 --d_model 320 --d_inner 576 --n_head 2 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-15 18:14:10.653149: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j16
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j16', experiment_name='config_0_j16', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/arch

Training baselines ( will take time). Once training is done, move the files from logdir to baseline_logs/hf_opt/wt2/ folder

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j1  --n_layer 1 --d_model 256 --d_inner 256 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 01:23:49.013320: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j1
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j1', experiment_name='config_0_j1', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j2  --n_layer 2 --d_model 256 --d_inner 256 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 01:27:47.724677: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j2
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j2', experiment_name='config_0_j2', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j3  --n_layer 3 --d_model 256 --d_inner 256 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 01:31:14.337811: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j3
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j3', experiment_name='config_0_j3', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j4  --n_layer 4 --d_model 256 --d_inner 256 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 01:34:38.963055: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j4
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j4', experiment_name='config_0_j4', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j5  --n_layer 5 --d_model 256 --d_inner 256 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 01:38:05.309087: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j5
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j5', experiment_name='config_0_j5', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j6  --n_layer 6 --d_model 256 --d_inner 256 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 01:41:34.221861: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j6
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j6', experiment_name='config_0_j6', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j7  --n_layer 7 --d_model 256 --d_inner 256 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 01:44:58.975767: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j7
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j7', experiment_name='config_0_j7', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/train.py --model_type hf_opt --config dgx1_1gpu_fp32 --config_file /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/wt103_base.yaml --max_step 680 --vocab word --experiment_name config_0_j8  --n_layer 8 --d_model 256 --d_inner 256 --n_head 4 --data "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext" --dataset wt2 --cache_dir "/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/cache" --batch_size 16

2023-06-16 01:48:28.800627: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Experiment dir : /content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j8
Namespace(work_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/logdir/config_0_j8', experiment_name='config_0_j8', append_dataset=False, append_time=False, cuda=True, fp16=False, restart='', pretrained_path='', debug=False, log_all_ranks=False, dllog_file='train_log.json', txtlog_file='train_log.log', save_all=False, no_env=True, no_train=False, no_eval=False, refresh_cache=False, log_interval=10, target_throughput=None, target_perplexity=None, apex_amp_opt_level='O2', affinity='socket_unique_interleaved', data='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-neurips-lts/archai/wikitext/textpred/wikitext-2', cache_dir='/content/drive/Shareddrives/ECE 226 Project/archai-neurips-lts/archai-

Plot pareto baseline - compare the validation perplexities of baseline and LTS. results are generated in logdir/hf_opt_titanxp_3d/pareto

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/search.py --plot_pareto_baseline --n_layer 2 3 4 5 6 7 8 --div_val 4 --n_head 2 4 8 --param_constraint_lower 25000 --param_constraint_upper 2500000 --device cuda --device_name titanxp --dataset wt2 --model_type hf_opt

2023-06-16 02:02:22.805504: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
{'config_0_j1_train_log.json': {'train_elapsed': 180.5124921798706, 'valid_ppl': 497.2821383410802, 'amlt_job': None}, 'config_0_j2_train_log.json': {'train_elapsed': 180.90603470802307, 'valid_ppl': 497.2821383410802, 'amlt_job': None}, 'config_0_j3_train_log.json': {'train_elapsed': 177.1047523021698, 'valid_ppl': 497.2821383410802, 'amlt_job': None}, 'config_0_j4_train_log.json': {'train_elapsed': 178.54214453697205, 'valid_ppl': 497.2821383410802, 'amlt_job': None}, 'config_0_j5_train_log.json': {'train_elapsed': 180.33936047554016, 'valid_ppl': 497.2821383410802, 'amlt_job': None}, 'config_0_j6_train_log.json': {'train_elapsed': 178.14144611358643, 'valid_ppl': 497.2821383410802, 'amlt_job': None}, 'config_0_j7_train_log.json': {'train_elapsed': 182.77853751182556, 'valid_ppl': 497.2821383410802, 'amlt_job': None}, 'config_0_j8_train_log.json': {'train_elapse

In [ ]:
!python /content/drive/Shareddrives/"ECE 226 Project"/archai-neurips-lts/archai-neurips-lts/archai/nlp/search.py --gen_tables --n_layer 2 3 4 5 6 7 8 --div_val 4 --n_head 2 4 8 --param_constraint_lower 25000 --param_constraint_upper 2500000 --device cuda --device_name titanxp --dataset wt2 --model_type hf_gpt2

2023-06-12 08:02:05.062996: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
table_titanxp.tex
{'vocab_size': 10000, 'n_positions': 192, 'n_embd': 128, 'd_embed': 512, 'n_inner': 256, 'resid_pdrop': 0.1, 'attn_pdrop': 0.0, 'initializer_range': 0.02, 'n_layer': 2, 'n_head': 2, 'd_head': 64, 'embd_pdrop': 0.0, 'tie_weight': True, 'activation_function': 'gelu_new', 'layer_norm_epsilon': 1e-05, 'summary_type': 'cls_index', 'summary_use_proj': True, 'summary_activation': None, 'summary_first_dropout': 0.1, 'summary_proj_to_labels': True, 'scale_attn_weights': True, 'use_cache': True, 'scale_attn_by_inverse_layer_idx': False, 'reorder_and_upcast_attn': False, 'bos_token_id': None, 'eos_token_id': None, 'return_dict': True, 'output_hidden_states': False, 'output_attentions': False, 'torchscript': False, 'torch_dtype': None, 'use_bfloat16': False, 'tf_legacy_loss': False, 'pruned_heads': {}, 'tie_word_embeddings': True, 'is_encoder_decoder': False